<a href="https://colab.research.google.com/github/abdullahmiraz/cnn-models-testing/blob/main/Fork_of_Citrus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'orange-leaf-disease-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3467863%2F6060073%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240607%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240607T150500Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7ebad48d23514961a024ebe360e8a5a4dfa82df4d6c1486e1889f0e94ecf4d38d83f32d6c56f47f21163de001c17076a52ea4dc4c03660dc028de2b72d2952b049edd6347f22a827d9054f64c7c50a7825f6f676d52c007d9139fcefc752d16475f6147bd8cee905e90026fe1e72fb9c71b01cbc40e334ebf9ae5749f3e09a4e9b90d8aee9de536e58498b7af2e0a31b223602643bdaf4597314ade868c08ed8903f6c7a5a8ac26e39fc05e52e3470c0373d7cff29d4b3ee23c36039ae61b38509fc145c89111eae9903f3552344b0a2006f20a1cf79de4e3744b0b899e3e20b846da577051d498f8701712b1217a4e1df9911544659f4f7e6581cd75b659105'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 261964789 bytes downloaded
Downloaded and uncompressed: orange-leaf-disease-dataset
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Canker_Diseases_Leaf_Orange/cropped_IMG_20230630_201659.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Canker_Diseases_Leaf_Orange/grayscale_IMG_20230630_201507.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Canker_Diseases_Leaf_Orange/sheared_IMG_20230630_201822_1.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Canker_Diseases_Leaf_Orange/shifted_IMG_20230630_202554_1.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Canker_Diseases_Leaf_Orange/shifted_IMG_20230630_201550.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Canker_Diseases_Leaf_Orange/sheared_IMG_20230630_202323.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Canker_Diseases_Leaf_Orange/sheared_IMG_20230630_202548.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Canker_Diseases_Leaf_Orange/inverte

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/shifted_IMG_20230630_200137_1.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/shifted_IMG_20230630_195545_1.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/IMG_20230630_200450.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/scaled_IMG_20230630_200344_1.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/IMG_20230630_195907.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/brightened_IMG_20230630_200357.jpg
/kaggle/input/orange-leaf-disease-dataset/train/train/Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/sharpened_IMG_20230630_201101.jpg
/kaggle/input/orange-leaf-disea

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import cv2
from sklearn.model_selection import train_test_split
import pandas as pd
import pathlib

# Define the path to the dataset
data_dir = '/kaggle/input/orange-leaf-disease-dataset/train/train'
data_dir = pathlib.Path(data_dir)

# Create a dictionary of class labels and corresponding image paths
objects = {
    'Citrus_Canker': list(data_dir.glob('Citrus_Canker_Diseases_Leaf_Orange/*')),
    'Nutrient_Deficiency': list(data_dir.glob('Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/*')),
    'Healthy_Leaf_Orange': list(data_dir.glob('Healthy_Leaf_Orange/*')),
    'Multiple_Diseases': list(data_dir.glob('Multiple_Diseases_Leaf_Orange/*')),
    'Young_Healthy': list(data_dir.glob('Young_Healthy_Leaf_Orange/*'))
}

# Define the class labels
objects_labels = {
    'Citrus_Canker': 0,
    'Nutrient_Deficiency': 1,
    'Healthy_Leaf_Orange': 2,
    'Multiple_Diseases': 3,
    'Young_Healthy': 4
}

# Create lists to store the image paths and labels
image_paths, y = [], []

# Collect the image paths and labels
for name, images in objects.items():
    for image in images:
        image_paths.append(str(image))
        y.append(objects_labels[name])

# Split the data into training and testing sets
image_paths_train, image_paths_test, y_train, y_test = train_test_split(image_paths, y, test_size=0.1, random_state=0)





# Create a DataFrame to store image paths and labels
train_df = pd.DataFrame({'image_path': image_paths_train, 'label': y_train})
test_df = pd.DataFrame({'image_path': image_paths_test, 'label': y_test})

# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Scale to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Scale to [0, 1]

# Set batch size and number of classes
batch_size = 32
num_classes = 5

# Create data iterators for training and testing
train_iterator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=True
)

test_iterator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False
)

# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    # Add more layers as needed
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Training the model
epochs = 30
model.fit(
    train_iterator,
    steps_per_epoch=len(train_df) // batch_size,
    epochs=epochs
)

# Evaluate the model on the test set
model.evaluate(
    test_iterator,
    steps=len(test_df) // batch_size
)

# Save the model
model.save('lung_cancer_classification_model.h5')


Found 34588 validated image filenames.
Found 3844 validated image filenames.
Epoch 1/30
1080/1080 [==============================] - 167s 154ms/step - loss: 0.9272 - accuracy: 0.6228
Epoch 2/30
 846/1080 [======================>.......] - ETA: 35s - loss: 0.6065 - accuracy: 0.7575

In [ ]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np

# Load the trained model
model = keras.models.load_model('/kaggle/input/citrus-disease-detection/lung_cancer_classification_model.h5')

# Function to preprocess the input image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))  # Resize the image to (64, 64)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Scale to [0, 1]
    return img

# Provide the path to the input image
input_image_path = '/kaggle/input/triall/Yellow.jpg'

# Preprocess the input image
input_image = preprocess_image(input_image_path)

# Make predictions using the loaded model
predictions = model.predict(input_image)

# Map the prediction to the corresponding class label
class_labels = ['Citrus_Canker', 'Nutrient_Deficiency', 'Healthy_Leaf_Orange', 'Multiple_Diseases', 'Young_Healthy']
predicted_class_index = np.argmax(predictions)
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class label:", predicted_class_label)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import cv2
from sklearn.model_selection import train_test_split
import pandas as pd
import pathlib

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Define the path to the dataset
data_dir = '/kaggle/input/orange-leaf-disease-dataset/train/train'
data_dir = pathlib.Path(data_dir)

# Create a dictionary of class labels and corresponding image paths
objects = {
    'Citrus_Canker': list(data_dir.glob('Citrus_Canker_Diseases_Leaf_Orange/*')),
    'Nutrient_Deficiency': list(data_dir.glob('Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/*')),
    'Healthy_Leaf_Orange': list(data_dir.glob('Healthy_Leaf_Orange/*')),
    'Multiple_Diseases': list(data_dir.glob('Multiple_Diseases_Leaf_Orange/*')),
    'Young_Healthy': list(data_dir.glob('Young_Healthy_Leaf_Orange/*'))
}

# Define the class labels
objects_labels = {
    'Citrus_Canker': 0,
    'Nutrient_Deficiency': 1,
    'Healthy_Leaf_Orange': 2,
    'Multiple_Diseases': 3,
    'Young_Healthy': 4
}

# Create lists to store the image paths and labels
image_paths, y = [], []

# Collect the image paths and labels
for name, images in objects.items():
    for image in images:
        image_paths.append(str(image))
        y.append(objects_labels[name])

# Split the data into training and testing sets
image_paths_train, image_paths_test, y_train, y_test = train_test_split(image_paths, y, test_size=0.1, random_state=0)

# Create a DataFrame to store image paths and labels
train_df = pd.DataFrame({'image_path': image_paths_train, 'label': y_train})
test_df = pd.DataFrame({'image_path': image_paths_test, 'label': y_test})

# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Scale to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Scale to [0, 1]

# Set batch size and number of classes
batch_size = 32
num_classes = 5

# Create data iterators for training and testing
train_iterator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=True
)

test_iterator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False
)

# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    # Add more layers as needed
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Training the model
epochs = 50
model.fit(
    train_iterator,
    steps_per_epoch=len(train_df) // batch_size,
    epochs=epochs
)

# Evaluate the model on the test set
model.evaluate(
    test_iterator,
    steps=len(test_df) // batch_size
)

# Save the model
model.save('lung_cancer_classification_model.h5')


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import cv2
from sklearn.model_selection import train_test_split
import pandas as pd
import pathlib

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Define the path to the dataset
data_dir = '/kaggle/input/orange-leaf-disease-dataset/train/train'
data_dir = pathlib.Path(data_dir)

# Create a dictionary of class labels and corresponding image paths
objects = {
    'Citrus_Canker': list(data_dir.glob('Citrus_Canker_Diseases_Leaf_Orange/*')),
    'Nutrient_Deficiency': list(data_dir.glob('Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/*')),
    'Healthy_Leaf_Orange': list(data_dir.glob('Healthy_Leaf_Orange/*')),
    'Multiple_Diseases': list(data_dir.glob('Multiple_Diseases_Leaf_Orange/*')),
    'Young_Healthy': list(data_dir.glob('Young_Healthy_Leaf_Orange/*'))
}

# Define the class labels
objects_labels = {
    'Citrus_Canker': 0,
    'Nutrient_Deficiency': 1,
    'Healthy_Leaf_Orange': 2,
    'Multiple_Diseases': 3,
    'Young_Healthy': 4
}

# Create lists to store the image paths and labels
image_paths, y = [], []

# Collect the image paths and labels
for name, images in objects.items():
    for image in images:
        image_paths.append(str(image))
        y.append(objects_labels[name])

# Split the data into training and testing sets
image_paths_train, image_paths_test, y_train, y_test = train_test_split(image_paths, y, test_size=0.1, random_state=0)

# Create a DataFrame to store image paths and labels
train_df = pd.DataFrame({'image_path': image_paths_train, 'label': y_train})
test_df = pd.DataFrame({'image_path': image_paths_test, 'label': y_test})

# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Scale to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Scale to [0, 1]

# Set batch size and number of classes
batch_size = 32
num_classes = 5

# Create data iterators for training and testing
train_iterator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=True
)

test_iterator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False
)

In [ ]:
# ... (Assuming you have already defined the necessary parts of the code)

# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Scale to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Scale to [0, 1]

# Set batch size and number of classes
batch_size = 32
num_classes = 5

# Create data iterators for training and testing
train_iterator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=True
)

test_iterator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False
)

# ... (Assuming you have defined and trained the model as well)

# Load the trained model
model = keras.models.load_model('/kaggle/input/citrus-disease-detection/lung_cancer_classification_model.h5')

# Evaluate the model on the test set
evaluation_result = model.evaluate(
    test_iterator,
    steps=len(test_df) // batch_size
)

# The evaluation_result contains the loss and metrics values
loss = evaluation_result[0]
accuracy = evaluation_result[1]

print("Test loss:", loss)
print("Test accuracy:", accuracy)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Load the trained model
model = keras.models.load_model('/kaggle/input/citrus-disease-detection/lung_cancer_classification_model.h5')

# Make predictions on the test set
y_pred = model.predict(test_iterator, steps=len(test_df) // batch_size)

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(test_df['label'], y_pred_labels)

print("Confusion Matrix:")
print(cm)
